In [225]:
import numpy as np
import math
np.set_printoptions(precision=128)

functionCalculated = 0

def f(x):
    x1 = x[0]
    x2 = x[1]
    global functionCalculated
    functionCalculated += 1
    # return x1**2 + x2**2
    # return (x1-1)**2 + 100*(x2-1)**2
    return 100 * (x1**2-x2)**2 + (x1-1)**2
    # return (100*(x1-x2)**2 + (x1-1)**2)
    # return (10*(x1-x2)**2 + (x1-1)**2)**4

In [226]:
# print(f"{h}, {h*np.array([1, 0])}, {h*np.array([0, 1])}")
def fderright(x, fx, h):
    gradFx1 = (f(x + h*np.array([1, 0])) - fx) / h
    gradFx2 = (f(x + h*np.array([0, 1])) - fx) / h
    return np.array([gradFx1, gradFx2])

def fderleft(x, fx, h):
    gradFx1 = (fx - f(x - h*np.array([1, 0]))) / h
    gradFx2 = (fx - f(x - h*np.array([0, 1]))) / h
    return np.array([gradFx1, gradFx2])

def fdercenter(x, h):
    gradFx1 = (f(x + h*np.array([1, 0])) - f(x - h*np.array([1, 0]))) / (2*h)
    gradFx2 = (f(x + h*np.array([0, 1])) - f(x - h*np.array([0, 1]))) / (2*h)
    return np.array([gradFx1, gradFx2])

In [227]:
def norm(x):
    x1 = x[0]
    x2 = x[1]
    return math.sqrt(x1**2 + x2**2)

def stopCriteria1(xk, fxk, xk_1, fxk_1, e):
    if ((norm(xk_1 - xk) / norm(xk)) <= e) and (math.fabs(fxk_1 - fxk) <= e):
        return True
    return False

def stopCriteria2(gradFxk, e):
    if (norm(gradFxk) <= e):
        return True
    return False

In [228]:
# Code for finding optimal Lambda
functionCalculatedForLambda = 0

svenCoefLambda = 0.001 # 0.00001 # 0.000098
l0 = 0

eGoldenCut = 0.001 # 0.1 # 0.11
eDSKPauela = 0.01

'''
def f(x):
    x1 = x[0]
    x2 = x[1]
    return 3*(x1-15)**  2 -x1*x2 +4*x2**2
'''

def FLambda(xk, l, Sk):
    global functionCalculatedForLambda
    functionCalculatedForLambda += 1
    return f(xk + l*Sk)

# trustInterval = sven(np.array([-22, -22]), np.array([0, 1]))
# print(goldenCut(trustInterval, np.array([-22, -22]), np.array([0, 1])))
# print(functionCalculatedForLambda)
    

In [229]:
def sven(xk, Sk):
    deltaL = svenCoefLambda * (norm(xk) / norm(Sk))
    FLambda0 = FLambda(xk, l0, Sk)
    FLambdaPlus = FLambda(xk, l0+deltaL, Sk)
    FLambdaMinus = FLambda(xk, l0-deltaL, Sk)
    
    coef = 0
    currentFValue = 0
    print(f"deltaL: {deltaL}")
    if FLambda0 < FLambdaPlus and FLambda0 < FLambdaMinus:
        print("Sven: both Lambdas are increasing funcion")
        return np.array([-deltaL, deltaL])
    if FLambda0 > FLambdaPlus and FLambda0 > FLambdaMinus:
        raise Exception("Sven: both Lambdas are deacreasing function")
    if FLambda0 > FLambdaPlus:
        coef = 1
        currentFValue = FLambdaPlus
        l1 = l0 + coef * deltaL
    if FLambda0 > FLambdaMinus:
        coef = -1
        currentFValue = FLambdaMinus
    l1 = l0 + coef * deltaL
    currentlValue = l1
    lambdaValues = [l0, l1]
    FNewLambda = 0
    n = 1
    while True:
        newLambda = currentlValue + 2*n*coef * deltaL
        lambdaValues.append(newLambda)
        FNewLambda = FLambda(xk, newLambda, Sk)
        if (currentFValue < FNewLambda):
            break
        currentlValue = newLambda
        currentFValue = FNewLambda
        n = 2*n
    print(f"SVEN - deltaL: {deltaL}, Lambdas: {lambdaValues}, functionCalculatedForLambda: {functionCalculatedForLambda}")
    return np.array([lambdaValues[-3], (lambdaValues[-2] + lambdaValues[-1]) / 2])

In [230]:
def goldenCut(lTrustInterval, xk, Sk):
    a = lTrustInterval[0]
    b = lTrustInterval[1]
    biggerCut = (math.sqrt(5) - 1) / 2
    smallerCut = 1 - biggerCut
    L = b - a
    x1 = a + smallerCut * L
    x2 = a + biggerCut * L
    Fx1 = FLambda(xk, x1, Sk)
    Fx2 = FLambda(xk, x2, Sk)

    while (L > eGoldenCut):
        if Fx1 < Fx2:
            b = x2
            x2 = x1
            L = b - a
            x1 = a + smallerCut * L
            Fx2 = Fx1
            Fx1 = FLambda(xk, x1, Sk)
        else:
            a = x1
            x1 = x2
            L = b - a
            x2 = a + biggerCut * L
            Fx1 = Fx2
            Fx2 = FLambda(xk, x2, Sk)
    print(f"GOLDENCUT - INTERVAL: {np.array([x1, x2])}, functionCalculatedForLambda: {functionCalculatedForLambda}")
    return np.array([x1, x2])

In [231]:
def dskStopCriteria(x2, Fx2, x_new, Fx_new):
    if (math.fabs(Fx2 - Fx_new) <= eDSKPauela) and (math.fabs(x2 - x_new) <= eDSKPauela):
        return True
    return False


# NOTE: OPTIMIZE DSKPAUELA
def DSKPauela(lTrustInterval, xk, Sk):
    # First iteration
    x1 = lTrustInterval[0]
    x2 = (lTrustInterval[0] + lTrustInterval[1]) / 2
    x3 = lTrustInterval[1]

    Fx1 = FLambda(xk, x1, Sk)
    Fx2 = FLambda(xk, x2, Sk)
    Fx3 = FLambda(xk, x3, Sk)

    deltaX = x2 - x1
    x_new = x2 + (deltaX * (Fx1 - Fx3)) / (2*(Fx1 - 2*Fx2 + Fx3))

    while True:
        Fx_new = FLambda(xk, x_new, Sk)
        if dskStopCriteria(x2, Fx2, x_new, Fx_new):
            print(f"DSKPAUELA - Lambda: {x_new}, functionCalculatedForLambda: {functionCalculatedForLambda}")
            return x_new
        
        if x1 < x_new and x_new < x2:
            x1 = x1
            x3 = x2
            x2 = x_new

            Fx1 = Fx1
            Fx2 = Fx_new
            Fx3 = Fx2
        elif x2 < x_new and x_new < x3:
            x1 = x2
            x3 = x3
            x2 = x_new

            Fx1 = Fx2
            Fx2 = Fx_new
            Fx3 = Fx3
        else:
            raise Exception(f"DSKPauela: can't choose points: x1: {x1} x2: {x2}, x3: {x3}, x_new: {x_new}")
        
        a1 = (Fx2 - Fx1) / (x2 - x1)
        a2 = (1 / (x3 - x2)) * ((Fx3 - Fx1) / (x3 - x1) - (Fx2 - Fx1) / (x2 - x1))
        x_new = (x1 + x2) / 2 - a1 / (2*a2)

In [232]:
x0 = np.array([-1.2, 0])

h = 0.1
def MOPlconst(e):
    xk = x0
    l = 1
    Fxk = f(xk)
    countGrad = True
    while True:
        if countGrad:
            gradFxk = fderright(xk, Fxk, h)# fdercenter(xk, h)
        xk_new = xk + l * (-gradFxk / norm(gradFxk))
        Fxk_new = f(xk_new)
        
        print(f'{xk_new}, {l}, {Fxk}, {Fxk_new}')
        
        if stopCriteria1(xk, Fxk, xk_new, Fxk_new, e):
            return xk_new
        if (Fxk <= Fxk_new):
            # What should I do with L?
            l = l / 2
            countGrad = False
        else: 
            xk = xk_new
            Fxk = Fxk_new
            countGrad = True

# stopCriteria2(gradFxk, e)
def MOPlopt(e):
    i = 0
    xk = x0
    Fxk = f(xk)
    gradFxk = fdercenter(xk, h) # fderleft(xk, Fxk, h)
    while True:
        #  fderright(xk, Fxk, h) fdercenter(xk, h)
        i += 1
        print(f"\nITERATION NUMBER {i}")
        Sk = - gradFxk
        l = goldenCut(sven(xk, Sk), xk, Sk) # np.mean(goldenCut(sven(xk, Sk), xk, Sk))
        xk_new = xk + l * Sk
        Fxk_new = f(xk_new)
        
        oldGrad = gradFxk
        gradFxk = fdercenter(xk_new, h) # fderleft(xk_new, Fxk_new, h)
        print(f'Xk: {xk}, Xk+1: {xk_new}, Lambda: {l}, Fxk: {Fxk}, Fxk+1: {Fxk_new}, GradFxk: {oldGrad}, GradFxk+1: {gradFxk}')
        
        if stopCriteria1(xk, Fxk, xk_new, Fxk_new, e): # stopCriteria1(xk, Fxk, xk_new, Fxk_new, e): # stopCriteria2(gradFxk, e):
            return xk_new
        xk = xk_new
        Fxk = Fxk_new

res = MOPlopt(0.01)
print(res)
print(functionCalculated)
# print(functionCalculatedForLambda)


ITERATION NUMBER 1
deltaL: 1.5845755840561636e-06
SVEN - deltaL: 1.5845755840561636e-06, Lambdas: [0, 1.5845755840561636e-06, 4.7537267521684905e-06, 1.1092029088393144e-05, 2.376863376084245e-05, 4.912184310574107e-05, 9.982826179553831e-05, 0.0002012410991751328, 0.0004040667739343217, 0.0008097181234526995, 0.0016210208224894554], functionCalculatedForLambda: 12
GOLDENCUT - INTERVAL: [0.0007139568298018357 0.0009054794171035633], functionCalculatedForLambda: 14
Xk: [-1.2  0. ], Xk+1: [-0.6999446364067937  0.2607780721258263], Lambda: [0.0007139568298018357 0.0009054794171035633], Fxk: 212.2, Fxk+1: 8.140528376040606, GradFxk: [-700.4000000000008 -288.0000000000001], GradFxk+1: [-70.35503144948417 -45.82888438176248]

ITERATION NUMBER 2
deltaL: 8.895918503366688e-06
SVEN - deltaL: 8.895918503366688e-06, Lambdas: [0, 8.895918503366688e-06, 2.6687755510100062e-05, 6.22714295235668e-05, 0.00013343877755050032, 0.0002757734736043673, 0.0005604428657121014, 0.0011297816499275695, 0.00226

KeyboardInterrupt: 